In [8]:
import time
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark_session = SparkSession\
            .builder\
            .master("spark://192.168.2.75:7077")\
            .appName("time_check")\
            .config("spark.executor.cores",4)\
            .getOrCreate()

spark_context = spark_session.sparkContext

df1 =  spark_session.read.json('hdfs://192.168.2.75:9000/reddit_data/500mb_data/RC_2010-11.bz2')

In [9]:
from re import sub

start_time = time.time()
df5 = df1.select("author","subreddit")\
        .groupBy("subreddit")\
        .count()\
        .orderBy(["count"],ascending=False)\
        .limit(5)

def map_values(subreddit,string):
    string_out = sub(r'[^A-Za-z0-9 \'/]+', u'', string.lower())
    string_out = string_out.split(' ')
    entry_list = []
    for entry in string_out:
        entry_list.append([(subreddit,entry),1])
    return (entry_list)

rdd1 = df5.join(df1.select("subreddit","body"),df5.subreddit==df1.subreddit)\
        .select(df1.subreddit,"body").rdd\
        .flatMap(lambda x: map_values(x[0],x[1]))\
        .reduceByKey(lambda x,y:x+y)\
        .sortBy(lambda x:x[1],ascending=False)

stop_time = time.time()
time_4_cores = stop_time - start_time
print (time_4_cores)

56.57413649559021


In [7]:
spark_session.stop()